# ***`Homework`***
---

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
jan23 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
feb23 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

df = pd.concat([jan23, feb23], ignore_index=True)
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,NaN
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,NaN
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,NaN
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,NaN
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,NaN


## ***Q1. Downloading the data***
---

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

- 16
- 17
- 18
- 19

In [10]:
print(f"Number of columns: {jan23.shape[1]}")

Number of columns: 19


## Q2. Computing duration
---

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

- 32.59
- 42.59
- 52.59
- 62.59

In [13]:
jan23['duration_minutes'] = (jan23['tpep_dropoff_datetime'] - jan23['tpep_pickup_datetime']).dt.total_seconds() / 60
jan23['duration_minutes'].describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration_minutes, dtype: float64

## ***Q3. Dropping outliers***
---

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

- 90%
- 92%
- 95%
- 98%

In [14]:
mask = (jan23['duration_minutes'] >= 1) & (jan23['duration_minutes'] <= 60)

# Filter out trips that are less than 1 minute and more than 60 minutes
jan23_clean = jan23[mask].copy()

# fraction
fraction = jan23_clean.shape[0] / jan23.shape[0]

print(f"Fraction of trips that are between 1 and 60 minutes: {fraction:.2f}")

Fraction of trips that are between 1 and 60 minutes: 0.98


## ***Q4. One-hot encoding***
---

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

- 2
- 155
- 345
- 515
- 715

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

# filter columns:
columns = ['PULocationID', 'DOLocationID', 'duration_minutes']
df = jan23_clean[columns].copy()

# cast to str
df['PULocationID'] = df['PULocationID'].astype(str)
df['DOLocationID'] = df['DOLocationID'].astype(str)

# split data
X = df.drop(columns='duration_minutes')
y = df['duration_minutes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# vectorize
dv = DictVectorizer(sparse=False)

train_dict      = X_train.to_dict(orient='records')
X_train_encoded = dv.fit_transform(train_dict)

test_dict       = X_test.to_dict(orient='records')
X_test_encoded  = dv.transform(test_dict)

# ncols
print(f"Number of columns: {X_train_encoded.shape[1]}")

Number of columns: 514


In [18]:
X_train_encoded[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## ***Q5. Training a model***
---

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?

- 3.64
- 7.64
- 11.64
- 16.64

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

rmse = lambda y_true, y_pred: np.sqrt( mean_squared_error(y_true, y_pred) )

# train model
model = LinearRegression()
model.fit(X_train_encoded, y_train)

# predict
y_pred_train = model.predict(X_train_encoded)

# evaluate
rmse(y_train, y_pred_train)

## ***Q6. Evaluating the model***
---

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

- 3.81
- 7.81
- 11.81
- 16.81

In [ ]:
y_pred_test = model.predict(X_test_encoded)
rmse(y_test, y_pred_test)